In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
print(torch.__version__)
print(torchvision.__version__)
import time

1.4.0
0.5.0


In [3]:
trans = []
resize =224
trans.append(torchvision.transforms.Resize(size=resize))
trans.append(torchvision.transforms.ToTensor())
transforms = torchvision.transforms.Compose(trans)

mnist_train = torchvision.datasets.FashionMNIST(root='/kaggle/data/FashionMNIST2065', train=True, download=True, transform=transforms)
mnist_test = torchvision.datasets.FashionMNIST(root='/kaggle/data/FashionMNIST2065', train=False,download=True, transform=transforms)

Extracting /kaggle/data/FashionMNIST2065/FashionMNIST/raw/train-images-idx3-ubyte.gz to /kaggle/data/FashionMNIST2065/FashionMNIST/raw


Extracting /kaggle/data/FashionMNIST2065/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /kaggle/data/FashionMNIST2065/FashionMNIST/raw


Extracting /kaggle/data/FashionMNIST2065/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /kaggle/data/FashionMNIST2065/FashionMNIST/raw


Extracting /kaggle/data/FashionMNIST2065/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /kaggle/data/FashionMNIST2065/FashionMNIST/raw
Processing...
Done!


In [4]:
len(mnist_train)

60000

In [33]:
#net
class Flatten(torch.nn.Module):  #展平操作
    def forward(self, x):
        return x.view(x.shape[0], -1)

class Reshape(torch.nn.Module): #将图像大小重定型
    def forward(self, x):
        return x.view(-1,1,28,28)      #(B x C x H x W)

def Lenet():
    net = torch.nn.Sequential(     #Lelet                                                  
        Reshape(),
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), #b*1*28*28  =>b*6*28*28
        nn.Sigmoid(),                                                       
        nn.AvgPool2d(kernel_size=2, stride=2),                              #b*6*28*28  =>b*6*14*14
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),           #b*6*14*14  =>b*16*10*10
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),                              #b*16*10*10  => b*16*5*5
        Flatten(),                                                          #b*16*5*5   => b*400
        nn.Linear(in_features=16*5*5, out_features=120),
        nn.Sigmoid(),
        nn.Linear(120, 84),
        nn.Dropout(0.5),
        nn.Sigmoid(),
        nn.Linear(84, 10)
        )
    return net

def Alexnet():
    net = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            Flatten(),
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            #由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )
    return net

def VGG():
    net = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            Flatten(),
            nn.Linear(in_features=3136, out_features=512, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=512, out_features=512, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=512, out_features=10, bias=True)
          )
    return net

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return nn.functional.avg_pool2d(x, kernel_size=x.size()[2:])
def NiN():
    net = nn.Sequential(
        nin_block(1, 96, kernel_size=11, stride=4, padding=0),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nin_block(96, 256, kernel_size=5, stride=1, padding=2),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nin_block(256, 384, kernel_size=3, stride=1, padding=1),
        nn.MaxPool2d(kernel_size=3, stride=2), 
        nn.Dropout(0.5),
        # 标签类别数是10
        nin_block(384, 10, kernel_size=3, stride=1, padding=1),
        GlobalAvgPool2d(), 
        # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
        Flatten())
    return net
net = Alexnet()

In [28]:
batch_size =128
train_iter =  torch.utils.data.DataLoader(dataset = mnist_train, batch_size = 256, shuffle = True)
test_iter = torch.utils.data.DataLoader(dataset = mnist_test, batch_size = 256, shuffle = True)

In [8]:
# This function has been saved in the d2l package for future use
#use GPU
def try_gpu():
    """If GPU is available, return torch.device as cuda:0; else return torch.device as cpu."""
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = try_gpu()
device

device(type='cuda', index=0)

In [9]:
def evaluate_accuracy(data_iter, net,device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    acc_sum,n = torch.tensor([0],dtype=torch.float32,device=device),0
    for X,y in data_iter:
        # If device is the GPU, copy the data to the GPU.
        X,y = X.to(device),y.to(device)
        net.eval()
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))  #[[0.2 ,0.4 ,0.5 ,0.6 ,0.8] ,[ 0.1,0.2 ,0.4 ,0.3 ,0.1]] => [ 4 , 2 ]
            n += y.shape[0]
    return acc_sum.item()/n

In [17]:
#训练函数
def train_ch5(net, train_iter, test_iter,criterion, num_epochs, batch_size, device,lr=None,weight_decay=None):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.Adam(params = net.parameters(), lr=lr,weight_decay = weight_decay)
    for epoch in range(num_epochs):
        train_l_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        train_acc_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        n, start = 0, time.time()
        for X, y in train_iter:
            net.train()
            
            optimizer.zero_grad()
            X,y = X.to(device),y.to(device) 
            y_hat = net(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                y = y.long()
                train_l_sum += loss.float()
                train_acc_sum += (torch.sum((torch.argmax(y_hat, dim=1) == y))).float()
                n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net,device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum/n, train_acc_sum/n, test_acc,
                 time.time() - start))

In [25]:
# 训练
lr, num_epochs,weight_decay = 0.001, 10,0.0001

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
net = net.to(device)

criterion = nn.CrossEntropyLoss()   #交叉熵描述了两个概率分布之间的距离，交叉熵越小说明两者之间越接近
train_ch5(net, train_iter, test_iter, criterion,num_epochs, batch_size,device, lr,weight_decay)

training on cuda:0
epoch 1, loss 0.0026, train acc 0.745, test acc 0.839, time 56.9 sec
epoch 2, loss 0.0013, train acc 0.874, test acc 0.890, time 58.0 sec
epoch 3, loss 0.0011, train acc 0.893, test acc 0.890, time 57.8 sec
epoch 4, loss 0.0010, train acc 0.906, test acc 0.898, time 56.3 sec
epoch 5, loss 0.0009, train acc 0.914, test acc 0.907, time 56.3 sec
epoch 6, loss 0.0009, train acc 0.919, test acc 0.904, time 57.6 sec
epoch 7, loss 0.0008, train acc 0.923, test acc 0.908, time 57.0 sec
epoch 8, loss 0.0008, train acc 0.929, test acc 0.908, time 56.7 sec
epoch 9, loss 0.0007, train acc 0.932, test acc 0.912, time 56.5 sec
epoch 10, loss 0.0007, train acc 0.936, test acc 0.902, time 56.9 sec


# First
    train acc 0.948, test acc 0.917,
    batch_size = 128
    GPU_max = 56
    time = 57
# Second
    train acc 0.917, test acc 0.900,
    batch_size = 128
    weight_decay = 0.01
    GPU_max = 50
    time = 57

# Third
    train acc 0.930, test acc 0.920,
    batch_size = 128
    weight_decay = 0.0001
    GPU_max = 50
    time = 57
# four
    train acc 0.936, test acc 0.902,
    batch_size = 32
    weight_decay = 0.0001
    GPU_max = 58
    time = 57


In [29]:
# 训练
lr, num_epochs,weight_decay = 0.001, 10,0.0001
net = VGG()
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
net = net.to(device)

criterion = nn.CrossEntropyLoss()   #交叉熵描述了两个概率分布之间的距离，交叉熵越小说明两者之间越接近
train_ch5(net, train_iter, test_iter, criterion,num_epochs, batch_size,device, lr,weight_decay)

training on cuda:0
epoch 1, loss 0.0028, train acc 0.734, test acc 0.867, time 48.8 sec
epoch 2, loss 0.0014, train acc 0.874, test acc 0.893, time 48.7 sec
epoch 3, loss 0.0011, train acc 0.895, test acc 0.907, time 48.7 sec
epoch 4, loss 0.0010, train acc 0.910, test acc 0.910, time 48.7 sec
epoch 5, loss 0.0009, train acc 0.920, test acc 0.919, time 48.0 sec
epoch 6, loss 0.0008, train acc 0.925, test acc 0.925, time 48.5 sec
epoch 7, loss 0.0007, train acc 0.932, test acc 0.926, time 48.4 sec
epoch 8, loss 0.0007, train acc 0.937, test acc 0.927, time 48.5 sec
epoch 9, loss 0.0006, train acc 0.941, test acc 0.925, time 48.5 sec
epoch 10, loss 0.0006, train acc 0.944, test acc 0.929, time 47.9 sec


# First
    train acc 0.942, test acc 0.921,
    batch_size = 32
    weight_decay = 0.0001
    GPU_max = 90
    time = 52
# second
    train acc 0.944, test acc 0.929,
    batch_size = 128
    weight_decay = 0.0001
    GPU_max = 90
    time = 48

In [37]:
# 训练
lr, num_epochs,weight_decay = 0.0001, 10,0.00001
net = NiN()
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
net = net.to(device)

criterion = nn.CrossEntropyLoss()   #交叉熵描述了两个概率分布之间的距离，交叉熵越小说明两者之间越接近
train_ch5(net, train_iter, test_iter, criterion,num_epochs, batch_size,device, lr,weight_decay)

training on cuda:0
epoch 1, loss 0.0071, train acc 0.372, test acc 0.574, time 47.8 sec
epoch 2, loss 0.0042, train acc 0.629, test acc 0.679, time 47.4 sec
epoch 3, loss 0.0035, train acc 0.688, test acc 0.700, time 47.9 sec
epoch 4, loss 0.0032, train acc 0.709, test acc 0.728, time 47.4 sec
epoch 5, loss 0.0030, train acc 0.729, test acc 0.735, time 47.7 sec
epoch 6, loss 0.0028, train acc 0.740, test acc 0.739, time 47.5 sec
epoch 7, loss 0.0027, train acc 0.748, test acc 0.753, time 46.9 sec
epoch 8, loss 0.0026, train acc 0.758, test acc 0.761, time 47.9 sec
epoch 9, loss 0.0025, train acc 0.762, test acc 0.768, time 47.8 sec
epoch 10, loss 0.0025, train acc 0.767, test acc 0.772, time 48.2 sec


# First
    train acc 0.864, test acc 0.860,
    lr =0.001
    batch_size = 128
    weight_decay = 0.0001
    GPU_max = 87
    time = 48
# Second
    train acc 0.767, test acc 0.772,
    lr = 0.01
    batch_size = 128
    weight_decay = 0.0001
    GPU_max = 87
    time = 48